In [1]:
import gdown

# download datasets for this exercise
dataset1_id = "1s2TkjSpzNc4SyxqRrQleZyDIHlc7bxnd"   # movies.csv
dataset2_id = "1Ws-_s1fHZ9nHfGLVUQurbHDvStePlEJm"   # directors.csv
dataset3_id = "173A59xh2mnpmljCCB9bhC4C5eP2IS6qZ"   # Pfizer_1.csv
output = "datasets/"

gdown.download(id=dataset1_id, output=output)
gdown.download(id=dataset2_id, output=output)
gdown.download(id=dataset3_id, output=output)

Downloading...
From: https://drive.google.com/uc?id=1s2TkjSpzNc4SyxqRrQleZyDIHlc7bxnd
To: /Users/home/DEV/scaler/python/public/class-sharing/datasets/movies.csv
100%|██████████| 112k/112k [00:00<00:00, 12.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Ws-_s1fHZ9nHfGLVUQurbHDvStePlEJm
To: /Users/home/DEV/scaler/python/public/class-sharing/datasets/directors.csv
100%|██████████| 65.4k/65.4k [00:00<00:00, 2.07MB/s]
Downloading...
From: https://drive.google.com/uc?id=173A59xh2mnpmljCCB9bhC4C5eP2IS6qZ
To: /Users/home/DEV/scaler/python/public/class-sharing/datasets/Pfizer_1.csv
100%|██████████| 1.51k/1.51k [00:00<00:00, 4.53MB/s]


'datasets/Pfizer_1.csv'

# DSML June24 Beginner / Pandas-4

## Agenda

1. Multi-Indexing
2. PFizer data
3. Melting
4. Pivoting
5. Pivot Table
6. Binning

## movies.csv and directors.csv

In [2]:
import pandas as pd

movies = pd.read_csv('datasets/movies.csv', index_col=0)
directors = pd.read_csv('datasets/directors.csv', index_col=0)

data = movies.merge(directors, how='left', left_on='director_id', right_on='id')
data.drop(['director_id','id_y'], axis=1, inplace=True)
data

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender
0,43597,237000000,150,2787965087,Avatar,7.2,11800,2009,Dec,Thursday,James Cameron,Male
1,43598,300000000,139,961000000,Pirates of the Caribbean: At World's End,6.9,4500,2007,May,Saturday,Gore Verbinski,Male
2,43599,245000000,107,880674609,Spectre,6.3,4466,2015,Oct,Monday,Sam Mendes,Male
3,43600,250000000,112,1084939099,The Dark Knight Rises,7.6,9106,2012,Jul,Monday,Christopher Nolan,Male
4,43602,258000000,115,890871626,Spider-Man 3,5.9,3576,2007,May,Tuesday,Sam Raimi,Male
...,...,...,...,...,...,...,...,...,...,...,...,...
1460,48363,0,3,321952,The Last Waltz,7.9,64,1978,May,Monday,Martin Scorsese,Male
1461,48370,27000,19,3151130,Clerks,7.4,755,1994,Sep,Tuesday,Kevin Smith,Male
1462,48375,0,7,0,Rampage,6.0,131,2009,Aug,Friday,Uwe Boll,Male
1463,48376,0,3,0,Slacker,6.4,77,1990,Jul,Friday,Richard Linklater,Male


In [3]:
data.groupby(['director_name'])['title'].count()

director_name
Adam McKay                      6
Adam Shankman                   8
Alejandro González Iñárritu     6
Alex Proyas                     5
Alexander Payne                 5
                               ..
Wes Craven                     10
Wolfgang Petersen               7
Woody Allen                    18
Zack Snyder                     7
Zhang Yimou                     6
Name: title, Length: 199, dtype: int64

In [4]:
data.groupby(['director_name'])['title'].count().sort_values(ascending = False)

director_name
Steven Spielberg    26
Clint Eastwood      19
Martin Scorsese     19
Woody Allen         18
Robert Rodriguez    16
                    ..
Paul Weitz           5
John Madden          5
Paul Verhoeven       5
John Whitesell       5
Kevin Reynolds       5
Name: title, Length: 199, dtype: int64

In [5]:
data_agg = data.groupby(['director_name'])[['year', 'title']].agg({'year': ['min', 'max'], 'title':'count'})
data_agg

year       title
                              min   max count
director_name                                
Adam McKay                   2004  2015     6
Adam Shankman                2001  2012     8
Alejandro González Iñárritu  2000  2015     6
Alex Proyas                  1994  2016     5
Alexander Payne              1999  2013     5
...                           ...   ...   ...
Wes Craven                   1984  2011    10
Wolfgang Petersen            1981  2006     7
Woody Allen                  1977  2013    18
Zack Snyder                  2004  2016     7
Zhang Yimou                  2002  2014     6

[199 rows x 3 columns]

In [6]:
data_agg["year"]

,min,max
director_name,,
Adam McKay,2004,2015
Adam Shankman,2001,2012
Alejandro González Iñárritu,2000,2015
Alex Proyas,1994,2016
Alexander Payne,1999,2013
...,...,...
Wes Craven,1984,2011
Wolfgang Petersen,1981,2006
Woody Allen,1977,2013


In [7]:
data_agg.columns

MultiIndex([( 'year',   'min'),
            ( 'year',   'max'),
            ('title', 'count')],
           )

In [8]:
# List comprehension method of generating new column names
new_column_names = ['_'.join(col) for col in data_agg.columns]
new_column_names

['year_min', 'year_max', 'title_count']

In [9]:
data_agg.columns = new_column_names
data_agg

,year_min,year_max,title_count
director_name,,,
Adam McKay,2004,2015,6
Adam Shankman,2001,2012,8
Alejandro González Iñárritu,2000,2015,6
Alex Proyas,1994,2016,5
Alexander Payne,1999,2013,5
...,...,...,...
Wes Craven,1984,2011,10
Wolfgang Petersen,1981,2006,7
Woody Allen,1977,2013,18


In [10]:
data_agg.reset_index()

,director_name,year_min,year_max,title_count
0,Adam McKay,2004,2015,6
1,Adam Shankman,2001,2012,8
2,Alejandro González Iñárritu,2000,2015,6
3,Alex Proyas,1994,2016,5
4,Alexander Payne,1999,2013,5
...,...,...,...,...
194,Wes Craven,1984,2011,10
195,Wolfgang Petersen,1981,2006,7
196,Woody Allen,1977,2013,18
197,Zack Snyder,2004,2016,7


In [11]:
data_agg['years_active'] = data_agg['year_max'] - data_agg['year_min']
data_agg.sort_values('years_active', ascending=False)

,year_min,year_max,title_count,years_active
director_name,,,,
Steven Spielberg,1977,2016,26,39
Martin Scorsese,1976,2013,19,37
Ridley Scott,1979,2015,16,36
Woody Allen,1977,2013,18,36
George Miller,1979,2015,7,36
...,...,...,...,...
Gary Winick,2002,2010,5,8
Jason Reitman,2005,2013,5,8
John Glen,1981,1989,5,8


In [12]:
data_agg['movies_per_year'] = data_agg['title_count']/data_agg['years_active']
data_agg.sort_values('movies_per_year', ascending=False)

,year_min,year_max,title_count,years_active,movies_per_year
director_name,,,,,
Tyler Perry,2006,2013,9,7,1.285714
Jason Friedberg,2006,2010,5,4,1.250000
Shawn Levy,2002,2014,11,12,0.916667
Robert Rodriguez,1992,2014,16,22,0.727273
Adam Shankman,2001,2012,8,11,0.727273
...,...,...,...,...,...
Lawrence Kasdan,1985,2012,5,27,0.185185
Luc Besson,1985,2014,5,29,0.172414
Robert Redford,1980,2010,5,30,0.166667


## Pfizer_1.csv

In [13]:
data = pd.read_csv('datasets/Pfizer_1.csv')
data

,Date,Drug_Name,Parameter,1:30:00,2:30:00,3:30:00,4:30:00,5:30:00,6:30:00,7:30:00,8:30:00,9:30:00,10:30:00,11:30:00,12:30:00
0,15-10-2020,diltiazem hydrochloride,Temperature,23.0,22.0,NaN,21.0,21.0,22,23.0,21.0,22.0,20,20.0,21
1,15-10-2020,diltiazem hydrochloride,Pressure,12.0,13.0,NaN,11.0,13.0,14,16.0,16.0,24.0,18,19.0,20
2,15-10-2020,docetaxel injection,Temperature,NaN,17.0,18.0,NaN,17.0,18,NaN,NaN,23.0,23,25.0,25
3,15-10-2020,docetaxel injection,Pressure,NaN,22.0,22.0,NaN,22.0,23,NaN,NaN,27.0,26,29.0,28
4,15-10-2020,ketamine hydrochloride,Temperature,24.0,NaN,NaN,27.0,NaN,26,25.0,24.0,23.0,22,21.0,20
5,15-10-2020,ketamine hydrochloride,Pressure,8.0,NaN,NaN,7.0,NaN,9,10.0,11.0,10.0,9,9.0,11
6,16-10-2020,diltiazem hydrochloride,Temperature,34.0,35.0,36.0,36.0,37.0,38,37.0,38.0,39.0,40,NaN,42
7,16-10-2020,diltiazem hydrochloride,Pressure,18.0,19.0,20.0,21.0,22.0,23,24.0,25.0,25.0,24,NaN,27
8,16-10-2020,docetaxel injection,Temperature,46.0,47.0,NaN,48.0,48.0,49,50.0,52.0,55.0,56,57.0,58
9,16-10-2020,docetaxel injection,Pressure,23.0,24.0,NaN,25.0,26.0,27,28.0,29.0,28.0,28,29.0,30


In [14]:
data_melt = data.melt(id_vars=['Date', 'Drug_Name', 'Parameter'],   # Columns to keep as identifiers
                      var_name='time',                              # Name for the new variable column
                      value_name='reading')                         # Name for the new value column
data_melt.head(30)

,Date,Drug_Name,Parameter,time,reading
0,15-10-2020,diltiazem hydrochloride,Temperature,1:30:00,23.0
1,15-10-2020,diltiazem hydrochloride,Pressure,1:30:00,12.0
2,15-10-2020,docetaxel injection,Temperature,1:30:00,NaN
3,15-10-2020,docetaxel injection,Pressure,1:30:00,NaN
4,15-10-2020,ketamine hydrochloride,Temperature,1:30:00,24.0
5,15-10-2020,ketamine hydrochloride,Pressure,1:30:00,8.0
6,16-10-2020,diltiazem hydrochloride,Temperature,1:30:00,34.0
7,16-10-2020,diltiazem hydrochloride,Pressure,1:30:00,18.0
8,16-10-2020,docetaxel injection,Temperature,1:30:00,46.0
9,16-10-2020,docetaxel injection,Pressure,1:30:00,23.0


In [15]:
data_tidy = data_melt.pivot(index=['Date','Drug_Name','Parameter'],    # Columns used to make new frame’s index
                            columns='time',                             # Column used to make new frame’s columns
                            values='reading')                           # Column used for populating new frame’s values.
data_tidy

time                                            10:30:00  11:30:00  12:30:00  \
Date       Drug_Name               Parameter                                   
15-10-2020 diltiazem hydrochloride Pressure         18.0      19.0      20.0   
                                   Temperature      20.0      20.0      21.0   
           docetaxel injection     Pressure         26.0      29.0      28.0   
                                   Temperature      23.0      25.0      25.0   
           ketamine hydrochloride  Pressure          9.0       9.0      11.0   
                                   Temperature      22.0      21.0      20.0   
16-10-2020 diltiazem hydrochloride Pressure         24.0       NaN      27.0   
                                   Temperature      40.0       NaN      42.0   
           docetaxel injection     Pressure         28.0      29.0      30.0   
                                   Temperature      56.0      57.0      58.0   
           ketamine hydrochloride  Pressure         16.0      17.0      18.0   
                                   Temperature      13.0      14.0      15.0   
17-10-2020 diltiazem hydrochloride Pressure         11.0      13.0      14.0   
                                   Temperature      14.0      11.0      10.0   
           docetaxel injection     Pressure         28.0      29.0      28.0   
                                   Temperature      21.0      22.0      23.0   
           ketamine hydrochloride  Pressure         13.0      14.0      15.0   
                                   Temperature      22.0      23.0      24.0   

time                                            1:30:00  2:30:00  3:30:00  \
Date       Drug_Name               Parameter                                
15-10-2020 diltiazem hydrochloride Pressure        12.0     13.0      NaN   
                                   Temperature     23.0     22.0      NaN   
           docetaxel injection     Pressure         NaN     22.0     22.0   
                                   Temperature      NaN     17.0     18.0   
           ketamine hydrochloride  Pressure         8.0      NaN      NaN   
                                   Temperature     24.0      NaN      NaN   
16-10-2020 diltiazem hydrochloride Pressure        18.0     19.0     20.0   
                                   Temperature     34.0     35.0     36.0   
           docetaxel injection     Pressure        23.0     24.0      NaN   
                                   Temperature     46.0     47.0      NaN   
           ketamine hydrochloride  Pressure        12.0     12.0     13.0   
                                   Temperature      8.0      9.0     10.0   
17-10-2020 diltiazem hydrochloride Pressure         3.0      4.0      4.0   
                                   Temperature     20.0     19.0     19.0   
           docetaxel injection     Pressure        20.0     22.0     22.0   
                                   Temperature     12.0     13.0     14.0   
           ketamine hydrochloride  Pressure         8.0      9.0     10.0   
                                   Temperature     13.0     14.0     15.0   

time                                            4:30:00  5:30:00  6:30:00  \
Date       Drug_Name               Parameter                                
15-10-2020 diltiazem hydrochloride Pressure        11.0     13.0     14.0   
                                   Temperature     21.0     21.0     22.0   
           docetaxel injection     Pressure         NaN     22.0     23.0   
                                   Temperature      NaN     17.0     18.0   
           ketamine hydrochloride  Pressure         7.0      NaN      9.0   
                                   Temperature     27.0      NaN     26.0   
16-10-2020 diltiazem hydrochloride Pressure        21.0     22.0     23.0   
                                   Temperature     36.0     37.0     38.0   
           docetaxel injection     Pressure        25.0     26.0     27.0   
           

In [16]:
data_tidy.index

MultiIndex([('15-10-2020', 'diltiazem hydrochloride',    'Pressure'),
            ('15-10-2020', 'diltiazem hydrochloride', 'Temperature'),
            ('15-10-2020',     'docetaxel injection',    'Pressure'),
            ('15-10-2020',     'docetaxel injection', 'Temperature'),
            ('15-10-2020',  'ketamine hydrochloride',    'Pressure'),
            ('15-10-2020',  'ketamine hydrochloride', 'Temperature'),
            ('16-10-2020', 'diltiazem hydrochloride',    'Pressure'),
            ('16-10-2020', 'diltiazem hydrochloride', 'Temperature'),
            ('16-10-2020',     'docetaxel injection',    'Pressure'),
            ('16-10-2020',     'docetaxel injection', 'Temperature'),
            ('16-10-2020',  'ketamine hydrochloride',    'Pressure'),
            ('16-10-2020',  'ketamine hydrochloride', 'Temperature'),
            ('17-10-2020', 'diltiazem hydrochloride',    'Pressure'),
            ('17-10-2020', 'diltiazem hydrochloride', 'Temperature'),
            ('17-10-

In [17]:
data_tidy.columns

Index(['10:30:00', '11:30:00', '12:30:00', '1:30:00', '2:30:00', '3:30:00',
       '4:30:00', '5:30:00', '6:30:00', '7:30:00', '8:30:00', '9:30:00'],
      dtype='object', name='time')

In [18]:
data_tidy.columns.name

'time'

In [19]:
data_tidy.reset_index()     # Original dataframe back

time,Date,Drug_Name,Parameter,10:30:00,11:30:00,12:30:00,1:30:00,2:30:00,3:30:00,4:30:00,5:30:00,6:30:00,7:30:00,8:30:00,9:30:00
0,15-10-2020,diltiazem hydrochloride,Pressure,18.0,19.0,20.0,12.0,13.0,NaN,11.0,13.0,14.0,16.0,16.0,24.0
1,15-10-2020,diltiazem hydrochloride,Temperature,20.0,20.0,21.0,23.0,22.0,NaN,21.0,21.0,22.0,23.0,21.0,22.0
2,15-10-2020,docetaxel injection,Pressure,26.0,29.0,28.0,NaN,22.0,22.0,NaN,22.0,23.0,NaN,NaN,27.0
3,15-10-2020,docetaxel injection,Temperature,23.0,25.0,25.0,NaN,17.0,18.0,NaN,17.0,18.0,NaN,NaN,23.0
4,15-10-2020,ketamine hydrochloride,Pressure,9.0,9.0,11.0,8.0,NaN,NaN,7.0,NaN,9.0,10.0,11.0,10.0
5,15-10-2020,ketamine hydrochloride,Temperature,22.0,21.0,20.0,24.0,NaN,NaN,27.0,NaN,26.0,25.0,24.0,23.0
6,16-10-2020,diltiazem hydrochloride,Pressure,24.0,NaN,27.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,25.0
7,16-10-2020,diltiazem hydrochloride,Temperature,40.0,NaN,42.0,34.0,35.0,36.0,36.0,37.0,38.0,37.0,38.0,39.0
8,16-10-2020,docetaxel injection,Pressure,28.0,29.0,30.0,23.0,24.0,NaN,25.0,26.0,27.0,28.0,29.0,28.0
9,16-10-2020,docetaxel injection,Temperature,56.0,57.0,58.0,46.0,47.0,NaN,48.0,48.0,49.0,50.0,52.0,55.0


In [20]:
# Pivot Example 1 – readings across time
data_pivot1 = data_melt.pivot(index=['Date', 'Drug_Name', 'Parameter'], columns='time', values='reading')
data_pivot1

time                                            10:30:00  11:30:00  12:30:00  \
Date       Drug_Name               Parameter                                   
15-10-2020 diltiazem hydrochloride Pressure         18.0      19.0      20.0   
                                   Temperature      20.0      20.0      21.0   
           docetaxel injection     Pressure         26.0      29.0      28.0   
                                   Temperature      23.0      25.0      25.0   
           ketamine hydrochloride  Pressure          9.0       9.0      11.0   
                                   Temperature      22.0      21.0      20.0   
16-10-2020 diltiazem hydrochloride Pressure         24.0       NaN      27.0   
                                   Temperature      40.0       NaN      42.0   
           docetaxel injection     Pressure         28.0      29.0      30.0   
                                   Temperature      56.0      57.0      58.0   
           ketamine hydrochloride  Pressure         16.0      17.0      18.0   
                                   Temperature      13.0      14.0      15.0   
17-10-2020 diltiazem hydrochloride Pressure         11.0      13.0      14.0   
                                   Temperature      14.0      11.0      10.0   
           docetaxel injection     Pressure         28.0      29.0      28.0   
                                   Temperature      21.0      22.0      23.0   
           ketamine hydrochloride  Pressure         13.0      14.0      15.0   
                                   Temperature      22.0      23.0      24.0   

time                                            1:30:00  2:30:00  3:30:00  \
Date       Drug_Name               Parameter                                
15-10-2020 diltiazem hydrochloride Pressure        12.0     13.0      NaN   
                                   Temperature     23.0     22.0      NaN   
           docetaxel injection     Pressure         NaN     22.0     22.0   
                                   Temperature      NaN     17.0     18.0   
           ketamine hydrochloride  Pressure         8.0      NaN      NaN   
                                   Temperature     24.0      NaN      NaN   
16-10-2020 diltiazem hydrochloride Pressure        18.0     19.0     20.0   
                                   Temperature     34.0     35.0     36.0   
           docetaxel injection     Pressure        23.0     24.0      NaN   
                                   Temperature     46.0     47.0      NaN   
           ketamine hydrochloride  Pressure        12.0     12.0     13.0   
                                   Temperature      8.0      9.0     10.0   
17-10-2020 diltiazem hydrochloride Pressure         3.0      4.0      4.0   
                                   Temperature     20.0     19.0     19.0   
           docetaxel injection     Pressure        20.0     22.0     22.0   
                                   Temperature     12.0     13.0     14.0   
           ketamine hydrochloride  Pressure         8.0      9.0     10.0   
                                   Temperature     13.0     14.0     15.0   

time                                            4:30:00  5:30:00  6:30:00  \
Date       Drug_Name               Parameter                                
15-10-2020 diltiazem hydrochloride Pressure        11.0     13.0     14.0   
                                   Temperature     21.0     21.0     22.0   
           docetaxel injection     Pressure         NaN     22.0     23.0   
                                   Temperature      NaN     17.0     18.0   
           ketamine hydrochloride  Pressure         7.0      NaN      9.0   
                                   Temperature     27.0      NaN     26.0   
16-10-2020 diltiazem hydrochloride Pressure        21.0     22.0     23.0   
                                   Temperature     36.0     37.0     38.0   
           docetaxel injection     Pressure        25.0     26.0     27.0   
           

In [21]:
data_pivot1.index.names

FrozenList(['Date', 'Drug_Name', 'Parameter'])

In [22]:
data_pivot1.columns.names

FrozenList(['time'])

In [23]:
data_pivot1.reset_index()

time,Date,Drug_Name,Parameter,10:30:00,11:30:00,12:30:00,1:30:00,2:30:00,3:30:00,4:30:00,5:30:00,6:30:00,7:30:00,8:30:00,9:30:00
0,15-10-2020,diltiazem hydrochloride,Pressure,18.0,19.0,20.0,12.0,13.0,NaN,11.0,13.0,14.0,16.0,16.0,24.0
1,15-10-2020,diltiazem hydrochloride,Temperature,20.0,20.0,21.0,23.0,22.0,NaN,21.0,21.0,22.0,23.0,21.0,22.0
2,15-10-2020,docetaxel injection,Pressure,26.0,29.0,28.0,NaN,22.0,22.0,NaN,22.0,23.0,NaN,NaN,27.0
3,15-10-2020,docetaxel injection,Temperature,23.0,25.0,25.0,NaN,17.0,18.0,NaN,17.0,18.0,NaN,NaN,23.0
4,15-10-2020,ketamine hydrochloride,Pressure,9.0,9.0,11.0,8.0,NaN,NaN,7.0,NaN,9.0,10.0,11.0,10.0
5,15-10-2020,ketamine hydrochloride,Temperature,22.0,21.0,20.0,24.0,NaN,NaN,27.0,NaN,26.0,25.0,24.0,23.0
6,16-10-2020,diltiazem hydrochloride,Pressure,24.0,NaN,27.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,25.0
7,16-10-2020,diltiazem hydrochloride,Temperature,40.0,NaN,42.0,34.0,35.0,36.0,36.0,37.0,38.0,37.0,38.0,39.0
8,16-10-2020,docetaxel injection,Pressure,28.0,29.0,30.0,23.0,24.0,NaN,25.0,26.0,27.0,28.0,29.0,28.0
9,16-10-2020,docetaxel injection,Temperature,56.0,57.0,58.0,46.0,47.0,NaN,48.0,48.0,49.0,50.0,52.0,55.0


In [25]:
# Pivot Table Example 1
data_pivot1.pivot_table(index='time', 
                        columns=['Drug_Name'], 
                        values=['Temperature', 'Pressure'],
                        aggfunc='max')


KeyError: 'Temperature'

In [ ]:
# Pivot Example 2 – readings across parameters
data_pivot2 = data_melt.pivot(index=['Date', 'time', 'Drug_Name'], columns='Parameter', values='reading')
data_pivot2.head(50)

In [ ]:
# Pivot Table Example 2
data_pivot2.pivot_table(index='Drug_Name', 
                        columns='Date', 
                        values=['Temperature', 'Pressure'], 
                        aggfunc='mean')

In [ ]:
# Pivot Table Example 2
data_melt.pivot_table(index=['time', 'Parameter'], 
                      columns='Drug_Name', 
                      values='reading', 
                      aggfunc=['min', 'max'])

In [ ]:
data_temp['Temperature'].min(axis=1)
# data_temp['Temperature'].max(axis=0)

In [ ]:
data_temp['mean_readings'] = data_temp['Temperature'].mean(axis=1)
data_temp

In [ ]:
temp_range = [15, 17, 19, 21, 23, 25, 60]
temp_labels = ['Low2', 'Low1', 'Low', 'Normal1', 'Normal2', 'Exceeds'] # labels define the severity of the resultant output of the test

data_temp['temp_range'] = pd.cut(temp_avg, bins=temp_range, labels=temp_labels)
data_temp

In [ ]:
data_temp['temp_range'].value_counts()